In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 70)
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn.preprocessing import LabelEncoder

%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Kaggle files from https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data

In [6]:
sales_train = pd.read_csv('./data/sales_train.csv')
test = pd.read_csv('./data/test.csv')
items = pd.read_csv('./data/items.csv')
item_categories = pd.read_csv('./data/item_categories.csv')
shops = pd.read_csv('./data/shops.csv')

In [7]:
items.columns = ['itemname','itemid','itemcategoryid']
item_categories.columns = ['itemcategoryname','itemcategoryid']
shops.columns = ['shopname','shopid']
sales_train.columns = ['date','month','shopid','itemid','price','target']
sales_train['date'] = pd.to_datetime(sales_train['date'],format='%d.%m.%Y')
sales_train = sales_train.sort_values(['date','month','shopid','itemid'])
test.columns = ['ID','shopid','itemid']

In [8]:
#Clean data, remove outliers, analysis taken from https://www.kaggle.com/dlarionov/feature-engineering-xgboost

In [9]:
sales_train = sales_train[sales_train.price<100000]
sales_train = sales_train[sales_train.target<1001]
median = sales_train[(sales_train.shopid==32)&(sales_train.itemid==2973)&(sales_train.month==4)&(sales_train.price>0)].price.median()
sales_train.loc[sales_train.price<0, 'price'] = median

sales_train.loc[sales_train.shopid == 0, 'shopid'] = 57
test.loc[test.shopid == 0, 'shopid'] = 57
sales_train.loc[sales_train.shopid == 1, 'shopid'] = 58
test.loc[test.shopid == 1, 'shopid'] = 58
sales_train.loc[sales_train.shopid == 10, 'shopid'] = 11
test.loc[test.shopid == 10, 'shopid'] = 11

In [7]:
#For each month, sum the daily sales for each shopid/itemid combination within this month
#That way, the training set will be similar to the test set
#The competion rules indicate that submitted target values should be clipped within the (0,20) range

In [8]:
targets = sales_train.groupby(by=['month','shopid','itemid']).agg({'target':'sum'}).reset_index()
targets['target'] = targets['target'].clip(0,20)
targets.head(2)

,month,shopid,itemid,target
0,0,2,27,1.0
1,0,2,33,1.0


In [9]:
sales = pd.DataFrame()
for month in range(34):#0..33
    salesmonth = sales_train[sales_train.month==month]
    shopidsdf = pd.DataFrame({'shopid':salesmonth.shopid.unique()}).sort_values('shopid').reset_index(drop=True)
    shopidsdf['key'] = 0
    itemidsdf = pd.DataFrame({'itemid':salesmonth.itemid.unique()}).sort_values('itemid').reset_index(drop=True)
    itemidsdf['key'] = 0
    salesmonth = pd.merge(shopidsdf,itemidsdf,how='outer',on='key')
    salesmonth['month'] = month
    salesmonth = salesmonth.drop(columns='key')
    sales = sales.append(salesmonth)
sales = sales[['month','shopid','itemid']].sort_values(by=['month','shopid','itemid'])
print(sales.shape)

(10913804, 3)


In [10]:
sales = pd.merge(sales,targets,on=['month','shopid','itemid'],how='left')
sales = sales.fillna(0)
sales = pd.merge(sales,items[['itemid','itemcategoryid']],on=['itemid'],how='left')
print(sales.shape)
sales.head(2)

(10913804, 5)


,month,shopid,itemid,target,itemcategoryid
0,0,2,19,0.0,40
1,0,2,27,1.0,19


In [11]:
del targets,salesmonth,itemidsdf,shopidsdf,median,month

In [13]:
#Append test set to training Dataset, so that the feature engineering applies to the test set as well

In [14]:
test['month'] = 34
test['target'] = 0
test = pd.merge(test,items[['itemid','itemcategoryid']],on=['itemid'],how='left')
sales = pd.concat([sales, test[sales.columns]], ignore_index=True)
sales.head(2).append(sales.tail(2))

,month,shopid,itemid,target,itemcategoryid
0,0,2,19,0.0,40
1,0,2,27,1.0,19
11128002,34,45,19648,0.0,40
11128003,34,45,969,0.0,37


In [15]:
#Shop infos, inspired by https://www.kaggle.com/kyakovlev/1st-place-solution-part-1-hands-on-data

In [16]:
shops['shopname'] = shops['shopname'].apply(lambda x: x.lower()).str.replace('[^\w\s]', '').str.replace('\d+','').str.strip()
shops['shopcity'] = shops['shopname'].str.partition(' ')[0]
shops['shoptype'] = shops['shopname'].apply(lambda x: 'мтрц' if 'мтрц' in x else 'трц' if 'трц' in x else 'трк' if 'трк' in x else 'тц' if 'тц' in x else 'тк' if 'тк' in x else 'NO_DATA')

encodercity = LabelEncoder()
shops['shopcityid'] = encodercity.fit_transform(shops['shopcity'])

encodertype = LabelEncoder()
shops['shoptypeid'] = encodertype.fit_transform(shops['shoptype'])

shops.head(1)

,shopname,shopid,shopcity,shoptype,shopcityid,shoptypeid
0,якутск орджоникидзе фран,0,якутск,NO_DATA,29,0


In [17]:
print(sales.shape)
sales = pd.merge(sales,shops[['shopid','shopcityid','shoptypeid']],how='left',on='shopid')
print(sales.shape)

(11128004, 5)
(11128004, 7)


In [18]:
#Extract type and subtype from category types (https://www.kaggle.com/dlarionov/feature-engineering-xgboost)

In [19]:
item_categories['split'] = item_categories['itemcategoryname'].str.split('-')

item_categories['itemcategorytype'] = item_categories['split'].map(lambda x: x[0].strip())
item_categories['itemcategorytypeid'] = LabelEncoder().fit_transform(item_categories['itemcategorytype'])

item_categories['itemcategorysubtype'] = item_categories['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
item_categories['itemcategorysubtypeid'] = LabelEncoder().fit_transform(item_categories['itemcategorysubtype'])

item_categories.head(3)

,itemcategoryname,itemcategoryid,split,itemcategorytype,itemcategorytypeid,itemcategorysubtype,itemcategorysubtypeid
0,PC - Гарнитуры/Наушники,0,"[PC , Гарнитуры/Наушники]",PC,0,Гарнитуры/Наушники,29
1,Аксессуары - PS2,1,"[Аксессуары , PS2]",Аксессуары,1,PS2,9
2,Аксессуары - PS3,2,"[Аксессуары , PS3]",Аксессуары,1,PS3,10


In [20]:
print(sales.shape)
sales = pd.merge(sales,item_categories[['itemcategoryid','itemcategorytypeid','itemcategorysubtypeid']],how='left',on='itemcategoryid')
print(sales.shape)

(11128004, 7)
(11128004, 9)


In [21]:
del encodercity, encodertype, shops, item_categories, items

In [23]:
#Time series of last 12 months as features

In [24]:
def timeserie(feature,sales):
    group = sales.groupby(['month',feature]).agg({'target':'mean'}).reset_index()
    for i in range(1,13):#1..12
        lagsdf = group.copy()
        lagsdf['month'] += i
        lagsdf.columns = ['month',feature,'ts'+feature+str(i)]
        sales = pd.merge(sales,lagsdf,how='left',on=['month',feature])
        sales['ts'+feature+str(i)] = sales['ts'+feature+str(i)].fillna(0)
        sales['ts'+feature+str(i)] = sales['ts'+feature+str(i)].astype(np.float16)
    return sales

In [25]:
for feature in ['itemid','shopid','shopcityid','shoptypeid','itemcategoryid','itemcategorytypeid','itemcategorysubtypeid']:
    print(feature)
    sales = timeserie(feature,sales)

itemid
shopid
shopcityid
shoptypeid
itemcategoryid
itemcategorytypeid
itemcategorysubtypeid


In [27]:
del feature

In [31]:
#Same with combinations of 2 categorical features (e.g. itemid/shopid)

In [32]:
def timeserie2features(feature1,feature2,sales):
    group = sales.groupby(['month',feature1,feature2]).agg({'target':'mean'}).reset_index()
    for i in range(1,13):#1..12
        lagsdf = group.copy()
        lagsdf['month'] += i
        lagsdf.columns = ['month',feature1,feature2,'ts'+feature1+feature2+str(i)]
        sales = pd.merge(sales,lagsdf,how='left',on=['month',feature1,feature2])
        sales['ts'+feature1+feature2+str(i)] = sales['ts'+feature1+feature2+str(i)].fillna(0)
        sales['ts'+feature1+feature2+str(i)] = sales['ts'+feature1+feature2+str(i)].astype(np.float16)
    return sales

In [33]:
featurescouples = [
    ['shopid','itemid'],
    ['itemcategoryid','itemcategorytypeid'],
    ['itemcategoryid','itemcategorysubtypeid'],
    ['itemcategoryid','shopcityid'],
    ['itemcategoryid','shoptypeid'],
    ['itemcategoryid','shopid'],
    ['itemcategorytypeid','itemcategorysubtypeid'],
    ['itemcategorytypeid','shopcityid'],
    ['itemcategorytypeid','shoptypeid'],
    ['itemcategorytypeid','shopid'],
    ['itemcategorysubtypeid','shopcityid'],
    ['itemcategorysubtypeid','shoptypeid'],
    ['itemcategorysubtypeid','shopid'],
    ['shopcityid','shoptypeid'],
    ['shopcityid','shopid'],
    ['shoptypeid','shopid']
]

In [34]:
for featurescouple in featurescouples:
    print(featurescouple)
    sales = timeserie2features(featurescouple[0],featurescouple[1],sales)

['shopid', 'itemid']
['itemcategoryid', 'itemcategorytypeid']
['itemcategoryid', 'itemcategorysubtypeid']
['itemcategoryid', 'shopcityid']
['itemcategoryid', 'shoptypeid']
['itemcategoryid', 'shopid']
['itemcategorytypeid', 'itemcategorysubtypeid']
['itemcategorytypeid', 'shopcityid']
['itemcategorytypeid', 'shoptypeid']
['itemcategorytypeid', 'shopid']
['itemcategorysubtypeid', 'shopcityid']
['itemcategorysubtypeid', 'shoptypeid']
['itemcategorysubtypeid', 'shopid']
['shopcityid', 'shoptypeid']
['shopcityid', 'shopid']
['shoptypeid', 'shopid']


In [35]:
#Means over all tagets within a month
group = sales.groupby(['month']).agg({'target':'mean'}).reset_index()
for i in range(1,13):#1..12
    lagsdf = group.copy()
    lagsdf['month'] += i
    lagsdf.columns = ['month','tstarget'+str(i)]
    print(lagsdf.columns)
    sales = pd.merge(sales,lagsdf,how='left',on=['month'])
    sales['tstarget'+str(i)] = sales['tstarget'+str(i)].fillna(0)
    sales['tstarget'+str(i)] = sales['tstarget'+str(i)].astype(np.float16)

Index(['month', 'tstarget1'], dtype='object')
Index(['month', 'tstarget2'], dtype='object')
Index(['month', 'tstarget3'], dtype='object')
Index(['month', 'tstarget4'], dtype='object')
Index(['month', 'tstarget5'], dtype='object')
Index(['month', 'tstarget6'], dtype='object')
Index(['month', 'tstarget7'], dtype='object')
Index(['month', 'tstarget8'], dtype='object')
Index(['month', 'tstarget9'], dtype='object')
Index(['month', 'tstarget10'], dtype='object')
Index(['month', 'tstarget11'], dtype='object')
Index(['month', 'tstarget12'], dtype='object')


In [37]:
del group, lagsdf, featurescouple, featurescouples, i

In [38]:
#Infer some features based on time series values

In [39]:
sales['meanlagitemshop'] = (sales['tsshopiditemid1'] + sales['tsshopiditemid2'] + sales['tsshopiditemid3']) / 3
sales['meanlagitem'] = (sales['tsitemid1'] + sales['tsitemid1'] + sales['tsitemid1']) / 3

In [40]:
#cumulative mean of shopid/item past values
sales_temp = sales[['month','shopid','itemid','target']].copy()
for i in range(1,35):#1..34
    lagsdf = sales[['month','shopid','itemid','target']].copy()
    lagsdf['month'] += i
    lagsdf.columns = ['month','shopid','itemid','tsshopiditemid'+str(i)]
    sales_temp = pd.merge(sales_temp,lagsdf,how='left',on=['month','shopid','itemid'])
    sales_temp['tsshopiditemid'+str(i)] = sales_temp['tsshopiditemid'+str(i)].fillna(0)
    sales_temp.loc[sales_temp.month == i, 'cumulativemeanitemshop'] = 0
    for j in range(1,i+1):
        sales_temp.loc[sales_temp.month == i, 'cumulativemeanitemshop'] += sales_temp.loc[sales_temp.month == i, 'tsshopiditemid'+str(j)]
    sales_temp.loc[sales_temp.month == i, 'cumulativemeanitemshop'] = sales_temp.loc[sales_temp.month == i, 'cumulativemeanitemshop'] / i

In [41]:
sales_temp = sales_temp[['month','shopid','itemid','cumulativemeanitemshop']]
print(sales.shape)
sales = pd.merge(sales,sales_temp,how='left',on=['month','shopid','itemid'])
print(sales.shape)

(11128004, 299)
(11128004, 300)


In [44]:
sales['cumulativemeanitemshop'] = sales['cumulativemeanitemshop'].fillna(0)

In [42]:
sales['deprecateditemshop'] = np.where(sales['meanlagitemshop'] == 0, 1, 0)
sales['deprecateditem'] = np.where(sales['meanlagitem'] == 0, 1, 0)

In [46]:
del lagsdf, sales_temp, i, j

In [47]:
#Date features

In [48]:
print(sales_train.date.min())
print(sales_train.date.max())

2013-01-01 00:00:00
2015-10-31 00:00:00


In [51]:
dates = pd.DataFrame()
dates['date'] = pd.date_range(datetime(2013,1,1),datetime(2015,11,30))
dates['year'] = dates.date.dt.year - 2013
dates['monthofyear'] = dates.date.dt.month
dates['dayofweek'] = dates.date.dt.dayofweek + 1
dates['month'] = dates.year * 12 + dates.monthofyear - 1
dates.head(2).append(dates.tail(2))

,date,year,monthofyear,dayofweek,month
0,2013-01-01,0,1,2,0
1,2013-01-02,0,1,3,0
1062,2015-11-29,2,11,7,34
1063,2015-11-30,2,11,1,34


In [52]:
nbweekends = dates[dates.dayofweek == 6].groupby('month').agg({'date':'count'}).reset_index()
nbweekends.columns = ['month','nbweekends']
datesdf = dates.groupby('month').agg({'date':'size','year':'mean','monthofyear':'mean'})
print(datesdf.shape)
datesdf = pd.merge(datesdf,nbweekends,on='month',how='left')
print(datesdf.shape)
datesdf.columns = ['month','nbdays','year','monthofyear','nbweekends']
datesdf

(35, 3)
(35, 5)


,month,nbdays,year,monthofyear,nbweekends
0,0,31,0,1,4
1,1,28,0,2,4
2,2,31,0,3,5
3,3,30,0,4,4
4,4,31,0,5,4
5,5,30,0,6,5
6,6,31,0,7,4
7,7,31,0,8,5
8,8,30,0,9,4
9,9,31,0,10,4


In [53]:
print(sales.shape)
sales = pd.merge(sales,datesdf,on='month',how='left')
print(sales.shape)

(11128004, 302)
(11128004, 306)


In [54]:
#Months since product launch

In [55]:
itemsales = sales.groupby(['itemid','month']).agg({'target':'sum'}).reset_index()
minmonths = itemsales.groupby('itemid').agg({'month':'min'}).reset_index()
minmonths.columns = ['itemid','minmonth']
itemsales = pd.merge(itemsales,minmonths,on='itemid',how='left')
itemsales['monthssinceitemlaunched'] = itemsales['month'] - itemsales['minmonth'] + 1
print(sales.shape)
sales = pd.merge(sales,itemsales[['itemid','month','monthssinceitemlaunched']],on=['itemid','month'],how='left')
print(sales.shape)

(11128004, 306)
(11128004, 307)


In [56]:
sales['newitem'] = np.where(sales.monthssinceitemlaunched==1,1,0)

In [57]:
sales.loc[sales.newitem==1,'deprecateditem'] = 0
sales.loc[sales.newitem==1,'deprecateditemshop'] = 0

In [59]:
del itemsales, minmonths, nbweekends, dates, datesdf

In [60]:
#Months since shop opened

In [61]:
shoplaunched = sales.groupby(['shopid','month']).agg({'target':'sum'}).reset_index()
minmonths = shoplaunched.groupby('shopid').agg({'month':'min'}).reset_index()
minmonths.columns = ['shopid','minmonth']
shoplaunched = pd.merge(shoplaunched,minmonths,on='shopid',how='left')
shoplaunched['monthssinceshoplaunched'] = shoplaunched['month'] - shoplaunched['minmonth'] + 1
shoplaunched = shoplaunched[['shopid','month','monthssinceshoplaunched']]
print(sales.shape)
sales = pd.merge(sales,shoplaunched,on=['shopid','month'],how='left')
print(sales.shape)

(11128004, 308)
(11128004, 309)


In [62]:
sales['newshop'] = 0
sales.loc[sales.monthssinceshoplaunched==1,'newshop'] = 1

In [63]:
sales.monthssinceitemlaunched.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35])

In [64]:
sales.monthssinceshoplaunched.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35])

In [65]:
#Price features

In [10]:
price_features = [c for c in sales.columns if 'price' in c]
sales = sales.drop(columns=price_features)

In [11]:
group = sales_train.groupby(['itemid']).agg({'price':['mean','min','max']}).reset_index()
group.columns = group.columns.get_level_values(0)
group.columns = ['itemid','itempriceavg','itempricemin','itempricemax']
sales = pd.merge(sales,group,on=['itemid'],how='left')
for c in ['itempriceavg','itempricemin','itempricemax']:
    sales[c] = sales[c].fillna(-1.0)
group = sales_train.groupby(['itemid','month']).agg({'price':'mean'}).reset_index()
sales = pd.merge(sales,group,on=['itemid','month'],how='left')
sales['price'] = sales['price'].fillna(-1.0)

In [12]:
def lastprice(month,itemid,lag):
    prices = group[(group.month < month)&(group.itemid==itemid)]['price']
    if len(prices) >= 2:
        return prices.values[-lag]
    elif len(prices) >= 1:
        return prices.values[-1]
    else:
        return -1.0

In [13]:
group = sales.groupby(['itemid','month']).agg({'price':'mean'}).reset_index()
group['lastitemprice'] = group.apply(lambda df: lastprice(df['month'],df['itemid'],1),axis=1)
group['antelastitemprice'] = group.apply(lambda df: lastprice(df['month'],df['itemid'],2),axis=1)
sales = pd.merge(sales,group[['itemid','month','lastitemprice','antelastitemprice']],on=['itemid','month'],how='left')

In [14]:
sales['ratiolastanteprice'] = 100 * sales['lastitemprice'] / sales['antelastitemprice']
sales['ratiolastavgprice'] = 100 * sales['lastitemprice'] / sales['itempriceavg']
sales['ratiolastminprice'] = 100 * sales['lastitemprice'] / sales['itempricemin']
sales['ratiolastmaxprice'] = 100 * sales['lastitemprice'] / sales['itempricemax']
sales['ratiominmaxprice'] = 100 * sales['itempricemax'] / sales['itempricemin']

In [15]:
sales.loc[sales.ratiolastanteprice<0,'ratiolastanteprice'] = 100.0
sales.loc[sales.ratiolastavgprice<0,'ratiolastavgprice'] = 100.0
sales.loc[sales.ratiolastminprice<0,'ratiolastminprice'] = 100.0
sales.loc[sales.ratiolastmaxprice<0,'ratiolastmaxprice'] = 100.0
sales.loc[sales.ratiominmaxprice<0,'ratiominmaxprice'] = 100.0

In [16]:
sales.loc[sales.newitem==1,'ratiominmaxprice'] = 100.0

In [17]:
sales[np.isinf(sales).any(axis=1)]

,month,shopid,itemid,target,itemcategoryid,shopcityid,shoptypeid,itemcategorytypeid,itemcategorysubtypeid,tsitemid1,tsitemid2,tsitemid3,tsitemid4,tsitemid5,tsitemid6,tsitemid7,tsitemid8,tsitemid9,tsitemid10,tsitemid11,tsitemid12,tsshopid1,tsshopid2,tsshopid3,tsshopid4,tsshopid5,tsshopid6,tsshopid7,tsshopid8,tsshopid9,tsshopid10,tsshopid11,tsshopid12,tsshopcityid1,tsshopcityid2,...,tsrevenue7,tsrevenue8,tsrevenue9,tsrevenue10,tsrevenue11,tsrevenue12,ratiorevenue12,itemno6,shopno6,tsshopiditemidproj2,tsshopiditemidproj3,tsshopiditemidproj6,tsshopiditemidproj12,tsitemidproj2,tsitemidproj3,tsitemidproj6,tsitemidproj12,tsshopidproj2,tsshopidproj3,tsshopidproj6,tsshopidproj12,meantsshopiditemidproj,meantsitemidproj,meantsshopidproj,itempriceavg,itempricemin,itempricemax,price,lastitemprice,antelastitemprice,ratiolastanteprice,ratiolastavgprice,ratiolastminprice,ratiolastmaxprice,ratiominmaxprice
365175,1,2,12,0.0,55,0,5,13,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.141235,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.141235,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.282471,0.211914,0.169556,0.154175,0.000000,0.000000,0.204590,189.000000,189.00,189.0,189.000000,-1.000000,-1.000000,100.000000,100.000000,100.000000,100.000000,100.000000
365176,1,2,27,0.0,19,0,5,5,10,0.155518,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.141235,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.141235,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,2.0,1.5,1.200195,1.090820,0.311035,0.233276,0.186646,0.169678,0.282471,0.211914,0.169556,0.154175,1.447266,0.225098,0.204590,1461.228571,498.00,2499.0,1998.000000,2325.000000,2325.000000,100.000000,159.112684,466.867470,93.037215,501.807229
365177,1,2,28,0.0,30,0,5,8,55,0.177734,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.141235,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.141235,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0.0,0.0,0.000000,0.000000,0.355469,0.266602,0.213257,0.193848,0.282471,0.211914,0.169556,0.154175,0.000000,0.257324,0.204590,310.010465,126.00,549.0,416.875000,549.000000,549.000000,100.000000,177.090796,435.714286,100.000000,435.714286
365178,1,2,30,0.0,40,0,5,11,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.141235,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.141235,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.282471,0.211914,0.169556,0.154175,0.000000,0.000000,0.204590,323.679206,99.00,399.0,383.921877,-1.000000,-1.000000,100.000000,100.000000,100.000000,100.000000,100.000000
365179,1,2,31,4.0,37,0,5,11,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.141235,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.141235,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.282471,0.211914,0.169556,0.154175,0.000000,0.000000,0.204590,578.630005,147.62,699.0,666.991044,-1.000000,-1.000000,100.000000,100.000000,100.000000,100.000000,100.000000
365180,1,2,32,0.0,40,0,5,11,4,5.378906,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.141235,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.141235,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0.0,0.0,0.000000,0.000000,10.757812,8.070312,6.457031,5.871094,0.282471,0.211914,0.169556,0.154175,0.000000,7.789062,0.204590,249.629

In [73]:
del c, minmonths, shoplaunched, group

In [74]:
#Shop Revenue features

In [18]:
revenue_features = [c for c in sales.columns if 'revenue' in c]
sales = sales.drop(columns=revenue_features)

In [19]:
sales_train['revenue'] = sales_train['price'] * sales_train['target']
group = sales_train.groupby(['month','shopid']).agg({'revenue':'sum'}).reset_index()

In [20]:
groupinitial = group.copy()
for i in range(1,13):#1..12
    lagsdf = groupinitial.copy()
    lagsdf['month'] += i
    lagsdf.columns = ['month','shopid','ts'+'revenue'+str(i)]
    print(lagsdf.columns)
    group = pd.merge(group,lagsdf,how='left',on=['month','shopid'])

Index(['month', 'shopid', 'tsrevenue1'], dtype='object')
Index(['month', 'shopid', 'tsrevenue2'], dtype='object')
Index(['month', 'shopid', 'tsrevenue3'], dtype='object')
Index(['month', 'shopid', 'tsrevenue4'], dtype='object')
Index(['month', 'shopid', 'tsrevenue5'], dtype='object')
Index(['month', 'shopid', 'tsrevenue6'], dtype='object')
Index(['month', 'shopid', 'tsrevenue7'], dtype='object')
Index(['month', 'shopid', 'tsrevenue8'], dtype='object')
Index(['month', 'shopid', 'tsrevenue9'], dtype='object')
Index(['month', 'shopid', 'tsrevenue10'], dtype='object')
Index(['month', 'shopid', 'tsrevenue11'], dtype='object')
Index(['month', 'shopid', 'tsrevenue12'], dtype='object')


In [21]:
group = group.drop(columns='revenue')
print(sales.shape)
sales = pd.merge(sales,group,on=['month','shopid'],how='left')
print(sales.shape)
for i in range(1,13):#1..12
    sales['tsrevenue'+str(i)] = sales['tsrevenue'+str(i)].fillna(0)

(11128004, 338)
(11128004, 350)


In [22]:
sales['ratiorevenue12'] = sales['tsrevenue1'] / sales['tsrevenue2']
sales['ratiorevenue12'] = sales['ratiorevenue12'].fillna(1.0).replace(np.inf, 1.0)

In [85]:
#Tag items with no sale for last 6 months

In [86]:
sales['itemno6'] = np.where(sales['tsitemid1']+sales['tsitemid2']+sales['tsitemid3']+sales['tsitemid4']+sales['tsitemid5']+sales['tsitemid6']==0,1,0)
sales.loc[sales.monthssinceitemlaunched <= 6,'itemno6'] = 0

In [87]:
#Same for shopid
sales['shopno6'] = np.where(sales['tsshopid1']+sales['tsshopid2']+sales['tsshopid3']+sales['tsshopid4']+sales['tsshopid5']+sales['tsshopid6']==0,1,0)
sales.loc[sales.monthssinceshoplaunched <= 6,'itemno6'] = 0

In [88]:
#Define some linear projections features + some means

In [89]:
sales['tsshopiditemidproj2'] = 2*(sales['tsshopiditemid1']-sales['tsshopiditemid2'])+sales['tsshopiditemid2']
sales['tsshopiditemidproj3'] = 3*(sales['tsshopiditemid1']-sales['tsshopiditemid3'])/2+sales['tsshopiditemid3']
sales['tsshopiditemidproj6'] = 6*(sales['tsshopiditemid1']-sales['tsshopiditemid6'])/5+sales['tsshopiditemid6']
sales['tsshopiditemidproj12'] = 12*(sales['tsshopiditemid1']-sales['tsshopiditemid12'])/11+sales['tsshopiditemid12']

sales['tsitemidproj2'] = 2*(sales['tsitemid1']-sales['tsitemid2'])+sales['tsitemid2']
sales['tsitemidproj3'] = 3*(sales['tsitemid1']-sales['tsitemid3'])/2+sales['tsitemid3']
sales['tsitemidproj6'] = 6*(sales['tsitemid1']-sales['tsitemid6'])/5+sales['tsitemid6']
sales['tsitemidproj12'] = 12*(sales['tsitemid1']-sales['tsitemid12'])/11+sales['tsitemid12']

sales['tsshopidproj2'] = 2*(sales['tsshopid1']-sales['tsshopid2'])+sales['tsshopid2']
sales['tsshopidproj3'] = 3*(sales['tsshopid1']-sales['tsshopid3'])/2+sales['tsshopid3']
sales['tsshopidproj6'] = 6*(sales['tsshopid1']-sales['tsshopid6'])/5+sales['tsshopid6']
sales['tsshopidproj12'] = 12*(sales['tsshopid1']-sales['tsshopid12'])/11+sales['tsshopid12']

sales['meantsshopiditemidproj'] = (sales['tsshopiditemidproj2']+sales['tsshopiditemidproj3']+sales['tsshopiditemidproj6']+sales['tsshopiditemidproj12'])/4
sales['meantsitemidproj'] = (sales['tsitemidproj2']+sales['tsitemidproj3']+sales['tsitemidproj6']+sales['tsitemidproj12'])/4
sales['meantsshopidproj'] = (sales['tsshopidproj2']+sales['tsshopidproj3']+sales['tsshopidproj6']+sales['tsshopidproj12'])/4

In [ ]:
#Save as a .csv file

In [24]:
sales.to_csv('sales.csv',index=False)

In [3]:
sales = pd.read_csv('sales.csv', nrows=100)

float_cols = [c for c in sales if (sales[c].dtype == "float64")&('revenue' not in c)&('ratio' not in c)]
float16_cols = {c: np.float16 for c in float_cols}

int_cols = [c for c in sales if (sales[c].dtype == "int64")&(c != 'itemid')]
int8_cols = {c: np.int8 for c in int_cols}

float16_cols.update(int8_cols)
sales = pd.read_csv('sales.csv', engine='c', dtype=float16_cols)